In [2]:
!pip3 install numpy opencv-python mediapipe tensorflow keras

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.


In [3]:
import cv2
import os

def extract_frames(video_path, output_folder):
    # Create output directory if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Capture the video
    cap = cv2.VideoCapture(video_path)
    count = 0

    # Loop through each frame
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Save the frame as an image file
        frame_filename = f"{output_folder}/frame_{count}.jpg"
        cv2.imwrite(frame_filename, frame)
        count += 1

    cap.release()
    print(f"Extracted {count} frames from {video_path}")

# Example usage
video_folder = "/Users/bugruster/Developer/MjrPrj/anksuchi"
output_base_folder = "/Users/bugruster/Developer/MjrPrj/anksuchi_frames_ext"

# Loop through each video file in the folder
for video_file in os.listdir(video_folder):
    video_path = os.path.join(video_folder, video_file)
    output_folder = os.path.join(output_base_folder, os.path.splitext(video_file)[0])

    extract_frames(video_path, output_folder)


Extracted 162 frames from /Users/bugruster/Developer/MjrPrj/anksuchi/IMG_3017.MOV.mp4.mp4
Extracted 189 frames from /Users/bugruster/Developer/MjrPrj/anksuchi/IMG_2702.MOV.mp4.mp4
Extracted 197 frames from /Users/bugruster/Developer/MjrPrj/anksuchi/IMG_2793.MOV.mp4.mp4
Extracted 232 frames from /Users/bugruster/Developer/MjrPrj/anksuchi/IMG_2698.MOV.mp4.mp4
Extracted 188 frames from /Users/bugruster/Developer/MjrPrj/anksuchi/IMG_2796.MOV.mp4.mp4
Extracted 156 frames from /Users/bugruster/Developer/MjrPrj/anksuchi/IMG_3012.MOV.mp4.mp4
Extracted 182 frames from /Users/bugruster/Developer/MjrPrj/anksuchi/IMG_2795.MOV.mp4.mp4
Extracted 214 frames from /Users/bugruster/Developer/MjrPrj/anksuchi/IMG_2701.MOV.mp4.mp4
Extracted 160 frames from /Users/bugruster/Developer/MjrPrj/anksuchi/IMG_3014.MOV.mp4.mp4
Extracted 238 frames from /Users/bugruster/Developer/MjrPrj/anksuchi/IMG_2697.MOV.mp4.mp4
Extracted 167 frames from /Users/bugruster/Developer/MjrPrj/anksuchi/IMG_3013.MOV.mp4.mp4
Extracted 

In [1]:
!pip3 install mediapipe

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import cv2
import mediapipe as mp
import os
import csv

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=2, min_detection_confidence=0.5)

# Initialize drawing utilities
mp_drawing = mp.solutions.drawing_utils

def process_frame(frame):
    # Convert the frame to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # Process the frame and detect hands
    result = hands.process(frame_rgb)

    landmarks = []
    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            # Extract and save landmarks
            for landmark in hand_landmarks.landmark:
                landmarks.append([landmark.x, landmark.y, landmark.z])
    return landmarks

def process_folder(input_folder, output_csv):
    with open(output_csv, mode='w', newline='') as file:
        writer = csv.writer(file)
        # Define CSV header
        header = ['frame'] + [f'hand_{i}_{axis}' for i in range(21) for axis in ['x', 'y', 'z']]
        writer.writerow(header)

        # Loop through each frame in the folder
        for frame_file in sorted(os.listdir(input_folder)):
            frame_path = os.path.join(input_folder, frame_file)
            frame = cv2.imread(frame_path)
            landmarks = process_frame(frame)

            # Write landmarks to CSV
            if landmarks:
                row = [frame_file] + [coord for landmark in landmarks for coord in landmark]
                writer.writerow(row)

# Example usage
input_base_folder = "/Users/bugruster/Developer/MjrPrj/anksuchi_frames_ext"
output_csv_folder = "/Users/bugruster/Developer/MjrPrj/anksuchi_landmarks_csv"

# Create the output folder if it doesn't exist
if not os.path.exists(output_csv_folder):
    os.makedirs(output_csv_folder)

# Process each folder containing frames
for folder in os.listdir(input_base_folder):
    input_folder = os.path.join(input_base_folder, folder)
    output_csv = os.path.join(output_csv_folder, f"{folder}_landmarks.csv")

    process_folder(input_folder, output_csv)


Matplotlib is building the font cache; this may take a moment.
/Users/bugruster/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
I0000 00:00:1724319456.358788  943277 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M1
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1724319456.365353  948549 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1724319456.376829  948549 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
/Users/bugruster/Library/Python/3.9/lib/python/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.G

In [4]:
!pip3 install pandas


Defaulting to user installation because normal site-packages is not writeable


In [34]:
import os
import pandas as pd

def load_data(csv_folder):
    data = []
    labels = []
    for csv_file in os.listdir(csv_folder):
        file_path = os.path.join(csv_folder, csv_file)
        try:
            # Attempt to read the CSV file
            df = pd.read_csv(file_path)
        except pd.errors.ParserError as e:
            # If there's an error, print which file caused it and continue
            print(f"Error parsing {csv_file}: {e}")
            continue
        # Extract the label from the CSV filename (e.g., "word_landmarks.csv")
        label = csv_file.split('_')[0]  # Adjust this based on your filename format
        for _, row in df.iterrows():
            # Extract the landmark data (skip the frame column)
            landmarks = row[1:].values
            data.append(landmarks)
            labels.append(label)
    return data, labels

# Run the function
csv_folder = "/Users/bugruster/Developer/MjrPrj/anksuchi_landmarks_csv"
data, labels = load_data(csv_folder)



In [35]:

import csv
import os

def clean_csv(file_path, expected_columns=64):
    cleaned_rows = []
    with open(file_path, 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            if len(row) == expected_columns:
                cleaned_rows.append(row)
            else:
                print(f"Skipping row with {len(row)} columns in {file_path}: {row}")

    # Write the cleaned data back to the file
    with open(file_path, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(cleaned_rows)

def clean_all_csv_files(csv_folder, expected_columns=64):
    for csv_file in os.listdir(csv_folder):
        file_path = os.path.join(csv_folder, csv_file)
        if csv_file.endswith('.csv'):
            clean_csv(file_path, expected_columns)

# Specify your folder path and clean the files
csv_folder = "/Users/bugruster/Developer/MjrPrj/anksuchi_landmarks_csv"
clean_all_csv_files(csv_folder)


In [36]:
import os
import pandas as pd
import numpy as np
def load_data(csv_folder):
    data = []
    labels = []
    for csv_file in os.listdir(csv_folder):
        file_path = os.path.join(csv_folder, csv_file)
        df = pd.read_csv(file_path)
        # Extract the label from the CSV filename (e.g., "word_landmarks.csv")
        label = csv_file.split('_')[0]  # Adjust this based on your filename format
        for _, row in df.iterrows():
            # Extract the landmark data (skip the frame column)
            landmarks = row[1:].values
            data.append(landmarks)
            labels.append(label)
    return np.array(data), np.array(labels)
csv_folder = "/Users/bugruster/Developer/MjrPrj/anksuchi_landmarks_csv"
data, labels = load_data(csv_folder)



In [37]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)


In [38]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)


In [39]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


In [40]:
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (1390, 63)
y_train shape: (1390, 1)
X_test shape: (348, 63)
y_test shape: (348, 1)


In [41]:
import os
import pandas as pd

csv_folder = "/Users/bugruster/Developer/MjrPrj/anksuchi_landmarks_csv"

# Initialize an empty list to store DataFrames
data_frames = []

# Loop through all CSV files in the folder
for csv_file in os.listdir(csv_folder):
    if csv_file.endswith('.csv'):
        file_path = os.path.join(csv_folder, csv_file)
        df = pd.read_csv(file_path)
        data_frames.append(df)

# Concatenate all DataFrames into one
data = pd.concat(data_frames, ignore_index=True)

print(data.head())  # To check the first few rows of the combined DataFrame

          frame  hand_0_x  hand_0_y      hand_0_z  hand_1_x  hand_1_y  \
0   frame_0.jpg  0.483951  0.839016 -5.859787e-08  0.503700  0.820380   
1   frame_1.jpg  0.483833  0.839386 -5.510154e-08  0.503567  0.820138   
2  frame_10.jpg  0.484048  0.839347 -5.571753e-08  0.503637  0.820481   
3  frame_11.jpg  0.484101  0.839273 -5.615657e-08  0.503601  0.820379   
4  frame_12.jpg  0.484157  0.839466 -5.777146e-08  0.503626  0.820351   

   hand_1_z  hand_2_x  hand_2_y  hand_2_z  ...  hand_17_z  hand_18_x  \
0  0.000056  0.521981  0.823995 -0.000427  ...  -0.002678   0.516552   
1 -0.000341  0.522027  0.823059 -0.000969  ...  -0.002772   0.517033   
2 -0.000713  0.522098  0.823942 -0.001490  ...  -0.002825   0.516845   
3 -0.000842  0.522052  0.823953 -0.001677  ...  -0.002930   0.516694   
4 -0.000598  0.522069  0.823759 -0.001354  ...  -0.002806   0.516359   

   hand_18_y  hand_18_z  hand_19_x  hand_19_y  hand_19_z  hand_20_x  \
0   0.928534  -0.001763   0.525772   0.930387   0.003068 

In [42]:
!pip3 install tensorflow

Defaulting to user installation because normal site-packages is not writeable


In [43]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import LabelEncoder

X_train = np.array(X_train, dtype='float32')
X_test = np.array(X_test, dtype='float32')

# Convert y_train and y_test to categorical if they are labels
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Fit the LabelEncoder on the training labels
y_train = np.argmax(y_train, axis=1)
y_test = np.argmax(y_test, axis=1)

# Update the code with the correct number of output classes
model = Sequential([
    Dense(128, input_shape=(X_train.shape[1],), activation='relu'),
    Dropout(0.3),  # To prevent overfitting
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(output_classes, activation='softmax')  # Output layer
])

# Display the model summary
model.summary()


/Users/bugruster/Library/Python/3.9/lib/python/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                │ (None, 128)            │         8,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 18,561 (72.50 KB)

 Trainable params: 18,561 (72.50 KB)

 Non-trainable params: 0 (0.00 B)

In [44]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [45]:
# print the shape of the input and output data
print(X_train.shape)
print(y_train.shape)


(1390, 63)
(1390,)


In [47]:
# Check the shape of y_train
print(y_train.shape)

# Reshape the output data
y_train = y_train.reshape(-1, 1)

# Print the new shape
print(y_train.shape)

(1390, 1)
(1390, 1)


In [52]:
# Convert y_train and y_test to categorical if they are labels
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Update the code with the correct number of output classes
output_classes = y_train.shape[1]

# Fit the LabelEncoder on the training labels
# Update the last layer of the model to have the correct number of units
model = Sequential([
    Dense(128, input_shape=(X_train.shape[1],), activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(output_classes, activation='softmax')  # Update the number of units
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])



/Users/bugruster/Library/Python/3.9/lib/python/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [55]:
# Reshape y_train to have the same rank as the output variable
y_train = y_train.reshape(-1, output_classes)


In [1]:
#train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

NameError: name 'model' is not defined

In [69]:
import os
import pandas as pd

csv_folder = "/Users/bugruster/Developer/MjrPrj/anksuchi_landmarks_csv"
data_frames = []

for csv_file in os.listdir(csv_folder):
    if csv_file.endswith('.csv'):
        file_path = os.path.join(csv_folder, csv_file)
        df = pd.read_csv(file_path)
        data_frames.append(df)

data = pd.concat(data_frames, ignore_index=True)

# Check the first few rows of the dataset to understand its structure
print(data.head())

# Check for NaN values in the dataset
print(data.isnull().sum())

# Check data types to ensure all columns are numeric except the label column
print(data.dtypes)



          frame  hand_0_x  hand_0_y      hand_0_z  hand_1_x  hand_1_y  \
0   frame_0.jpg  0.483951  0.839016 -5.859787e-08  0.503700  0.820380   
1   frame_1.jpg  0.483833  0.839386 -5.510154e-08  0.503567  0.820138   
2  frame_10.jpg  0.484048  0.839347 -5.571753e-08  0.503637  0.820481   
3  frame_11.jpg  0.484101  0.839273 -5.615657e-08  0.503601  0.820379   
4  frame_12.jpg  0.484157  0.839466 -5.777146e-08  0.503626  0.820351   

   hand_1_z  hand_2_x  hand_2_y  hand_2_z  ...  hand_17_z  hand_18_x  \
0  0.000056  0.521981  0.823995 -0.000427  ...  -0.002678   0.516552   
1 -0.000341  0.522027  0.823059 -0.000969  ...  -0.002772   0.517033   
2 -0.000713  0.522098  0.823942 -0.001490  ...  -0.002825   0.516845   
3 -0.000842  0.522052  0.823953 -0.001677  ...  -0.002930   0.516694   
4 -0.000598  0.522069  0.823759 -0.001354  ...  -0.002806   0.516359   

   hand_18_y  hand_18_z  hand_19_x  hand_19_y  hand_19_z  hand_20_x  \
0   0.928534  -0.001763   0.525772   0.930387   0.003068 

In [70]:
# Fill NaN values with 0 (or use a different strategy if needed)
data = data.fillna(0)

# Convert all feature columns to numeric types
for col in data.columns:
    if data[col].dtype == 'object':
        try:
            data[col] = pd.to_numeric(data[col])
        except ValueError:
            print(f"Column {col} cannot be converted to numeric.")

# Check again after conversion
print(data.dtypes)


Column frame cannot be converted to numeric.
frame         object
hand_0_x     float64
hand_0_y     float64
hand_0_z     float64
hand_1_x     float64
              ...   
hand_19_y    float64
hand_19_z    float64
hand_20_x    float64
hand_20_y    float64
hand_20_z    float64
Length: 64, dtype: object


In [71]:
# Print column names to identify the label column
print(data.columns)

# Replace 'sign_class' with the correct label column name
label_column = 'sign_class'  # Update this if your label column is named differently


Index(['frame', 'hand_0_x', 'hand_0_y', 'hand_0_z', 'hand_1_x', 'hand_1_y',
       'hand_1_z', 'hand_2_x', 'hand_2_y', 'hand_2_z', 'hand_3_x', 'hand_3_y',
       'hand_3_z', 'hand_4_x', 'hand_4_y', 'hand_4_z', 'hand_5_x', 'hand_5_y',
       'hand_5_z', 'hand_6_x', 'hand_6_y', 'hand_6_z', 'hand_7_x', 'hand_7_y',
       'hand_7_z', 'hand_8_x', 'hand_8_y', 'hand_8_z', 'hand_9_x', 'hand_9_y',
       'hand_9_z', 'hand_10_x', 'hand_10_y', 'hand_10_z', 'hand_11_x',
       'hand_11_y', 'hand_11_z', 'hand_12_x', 'hand_12_y', 'hand_12_z',
       'hand_13_x', 'hand_13_y', 'hand_13_z', 'hand_14_x', 'hand_14_y',
       'hand_14_z', 'hand_15_x', 'hand_15_y', 'hand_15_z', 'hand_16_x',
       'hand_16_y', 'hand_16_z', 'hand_17_x', 'hand_17_y', 'hand_17_z',
       'hand_18_x', 'hand_18_y', 'hand_18_z', 'hand_19_x', 'hand_19_y',
       'hand_19_z', 'hand_20_x', 'hand_20_y', 'hand_20_z'],
      dtype='object')


In [73]:
# Drop the label column to get the features
X = data.drop(columns=[label_column])

# Encode the labels if they are not already numeric
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data[label_column])

# Ensure X is numeric
X = X.astype(float)

# Normalize the features
X = X / X.max()

print(X.head())
print(y[:5])


KeyError: "['sign_class'] not found in axis"